In [132]:
import numpy as np
from typing import Callable, Tuple
from sklearn.model_selection import KFold
from sklearn.linear_model import ElasticNet
from tqdm import tqdm

kf = KFold(n_splits=10)

In [133]:
Kernel = Callable[[np.ndarray], np.ndarray]


KERNEL: Kernel = lambda x: np.array([
  x[0],
  x[1],
  x[2],
  x[3],
  x[4],
  x[0] ** 2,
  x[1] ** 2,
  x[2] ** 2,
  x[3] ** 2,
  x[4] ** 2,
  np.exp(x[0]),
  np.exp(x[1]),
  np.exp(x[2]),
  np.exp(x[3]),
  np.exp(x[4]),
  np.cos(x[0]),
  np.cos(x[1]),
  np.cos(x[2]),
  np.cos(x[3]),
  np.cos(x[4]),
  1,
])

In [134]:
def compute_features(kernel: Kernel, X: np.ndarray) -> np.ndarray:
  return np.apply_along_axis(kernel, 1, X)

In [135]:
data = np.genfromtxt('train.csv', delimiter=',')
y = data[1:,1]
X = data[1:,2:]

In [136]:
def predict(w: np.ndarray, X: np.ndarray) -> np.ndarray:
  return X @ w

In [137]:
def rmse(y_test: np.ndarray, y_pred: np.ndarray) -> float:
  mse = np.square(np.subtract(y_test, y_pred)).mean()
  return np.sqrt(mse)

In [138]:
def cv(lambd: float, X: np.ndarray, y: np.ndarray) -> Tuple[np.ndarray, float]:
  results = np.array([])

  X_feat = compute_features(KERNEL, X)

  for train_index, test_index in kf.split(X_feat):
    X_train, X_test, y_train, y_test = X_feat[train_index], X_feat[test_index], y[train_index], y[test_index]
    regr = ElasticNet(random_state=0, alpha=lambd, fit_intercept=False)
    regr.fit(X_train, y_train)
    y_pred = predict(regr.coef_, X_test)
    result = rmse(y_test, y_pred)
    results = np.append(results, [result])

  return regr, results.mean()

In [139]:
lambds = [0.01, 0.1, 1, 10, 100, 200]

loss_opt = np.inf
for lambd in tqdm(lambds):
    regr_new, loss = cv(lambd, X, y)
    if loss < loss_opt:
        loss_opt = loss
        regr = regr_new
loss_opt

  0%|          | 0/6 [00:00<?, ?it/s]/Users/jonas/Projects/school/iml/venv/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.281e+01, tolerance: 2.736e+00
  model = cd_fast.enet_coordinate_descent(
/Users/jonas/Projects/school/iml/venv/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.010e+01, tolerance: 2.747e+00
  model = cd_fast.enet_coordinate_descent(
/Users/jonas/Projects/school/iml/venv/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iteration

1.9521189464764443

In [141]:
y_pred = predict(regr.coef_, compute_features(KERNEL, X))
rmse(y, y_pred)

1.9568775769845357

In [140]:
output = regr.coef_
np.savetxt('output.csv', output)